In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.insert(0, "../../code-previous")

In [3]:
import pandas as pd
import numpy as np
import networkx as nx
import sklearn.feature_extraction as skfeatures
import time
import os

import operator

import string
import swifter
import nltk
stopwords = nltk.corpus.stopwords.words('english')


import plotly
import plotly.plotly as py
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='sreejith_s', api_key='rcet0rTqFCc3WeOzWTjv')



from sklearn.feature_extraction.text import CountVectorizer


from unicodedata import category
from tqdm import tqdm
tqdm.pandas(desc="Progress")

import seaborn as sns
sns.set_style('white')
import matplotlib.pylab as plt

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
data_path = "/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/articles.pql"
article_df = pd.read_pickle(data_path)

In [5]:
data_path = "/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/biology.pql"
data = None
bio_categories = pd.read_pickle(data_path)

In [6]:
data = None
data = bio_categories.merge(article_df, 
                            how = 'inner',
                            on = 'ArticleID')

In [7]:
translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))

In [8]:
data['title_without_punct'] = data['Title'].swifter.apply(lambda x: x.lower().translate(translator))

Pandas Apply: 100%|██████████| 9446057/9446057 [00:41<00:00, 229929.09it/s]


In [9]:
data["title_without_stopwords"] = data['title_without_punct'].swifter.apply(lambda x: " ".join([word for word in x.split() if word not in stopwords and not word.isdigit() and len(word) >= 4]))

Pandas Apply: 100%|██████████| 9446057/9446057 [04:21<00:00, 36120.58it/s]


In [10]:
pd.set_option('display.max_colwidth', -1)

In [11]:
data[["Title","title_without_stopwords"]].head()

,Title,title_without_stopwords
0,ON THE CONSTRUCTION OF NANOSECOND TIME-RESOLVED EMISSION-SPECTRA,construction nanosecond time resolved emission spectra
1,ON THE CONSTRUCTION OF NANOSECOND TIME-RESOLVED EMISSION-SPECTRA,construction nanosecond time resolved emission spectra
2,ON THE CONSTRUCTION OF NANOSECOND TIME-RESOLVED EMISSION-SPECTRA,construction nanosecond time resolved emission spectra
3,ON THE CONSTRUCTION OF NANOSECOND TIME-RESOLVED EMISSION-SPECTRA,construction nanosecond time resolved emission spectra
4,SEROTONIN (5-HYDROXYTRYPTAMINE) - A POSSIBLE REGULATOR OF GLYCOGENOLYSIS IN PERFUSED MUSCLE SEGMENTS OF ASCARIS-SUUM,serotonin hydroxytryptamine possible regulator glycogenolysis perfused muscle segments ascaris suum


In [ ]:
data = data.sort_values(by="PubYear")

In [ ]:
vectorizer = CountVectorizer()
vectorized_words = vectorizer.fit_transform(data.title_without_stopwords)

In [ ]:
row_counts = data.groupby('PubYear').size().reset_index(name="count")

In [ ]:
counts =  list(row_counts["count"])

__What are the overall topwords__

In [ ]:
N = 50

In [ ]:
vocab = vectorizer.vocabulary_
rev_vocab = {index:word for word,index in vocab.items()}

In [ ]:
# Highest Frequency words all time
all_time_word_frequencies = vectorized_words.sum(axis=0).reshape(-1,).tolist()[0]
word_index_with_freq = {index:freq for index,freq in enumerate(all_time_word_frequencies)}

In [ ]:
word_with_freq = {rev_vocab[index]:np.log(freq + 1) for index, freq in word_index_with_freq.items()}

In [ ]:
sorted_word_with_freq = sorted(word_with_freq.items(), key=operator.itemgetter(1), reverse=True)

In [ ]:
top_words_all_corpus = pd.DataFrame(list(sorted_word_with_freq[:50]), columns=["word","frequency"])

In [ ]:
datatoplot = [go.Line(
            x=top_words_all_corpus["word"],
            y=top_words_all_corpus["frequency"]
    )]



layout = go.Layout(
    title='Top Frequency Words',
    
    xaxis=dict(
        title='Word',
        titlefont=dict(
            family='Lato',
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Frequency',
        titlefont=dict(
            family='Lato',
            color='#7f7f7f'
        )
    )
)

fig = go.Figure(data=datatoplot, layout=layout)
py.iplot(fig, filename='freq')

/usr/local/lib/python3.6/dist-packages/plotly/graph_objs/_deprecations.py:372: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [ ]:
index_start = 0
yearly_top_words = []

N = 5000

for i in range(len(row_counts)):
    #print(i)
    offset = index_start + counts[i]
    tmp = vectorized_words[index_start:offset,:]
    yearly_word_freq = tmp.sum(axis=0).reshape(-1,).tolist()[0]
    
    word_index_with_freq = {index:freq for index,freq in enumerate(yearly_word_freq)}
    word_with_freq = {rev_vocab[index]:np.log(freq + 1) for index, freq in word_index_with_freq.items()}
    sorted_word_with_freq = sorted(word_with_freq.items(), key=operator.itemgetter(1), reverse=True)
    top_words = [word for (word,freq) in sorted_word_with_freq[:N]]
    yearly_top_words.append(top_words)

In [ ]:
years = list(range(1901, 2017))
overlapping_word_counts = []

for i in range(1, len(row_counts)):
    
    overlap_count = len([word for word in yearly_top_words[i] if word in yearly_top_words[i-1]])
    overlapping_word_counts.append(overlap_count)

__How many top words are overlapping with the words from adjacent year__

In [ ]:
datatoplot = [go.Line(
            x=years,
            y=overlapping_word_counts
    )]



layout = go.Layout(
    title='Overlap',
    
    xaxis=dict(
        title='Word',
        titlefont=dict(
            family='Lato',
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Overlap Count',
        titlefont=dict(
            family='Lato',
            color='#7f7f7f'
        ),
        range=[0, 5000]
    )
)

fig = go.Figure(data=datatoplot, layout=layout)
py.iplot(fig, filename='freq')